# The random phase approximation (RPA)


TO BE WRITTEN : (material below)


- Recall the formula for RPA


- TODO : 
 
 - we grab chi(q,omega)
-------------------------------------------


The Lindhardt susceptibility $\chi_0$ is the exeact susceptibility for the non-interacting case $U=0$, however for finite interactions the susceptibility of the system $\chi$ is given by the Behte-Salpeter equation

$$ \chi = \chi_0 + \chi_0 \Gamma \chi $$

where $\Gamma$ is the particle-hole irreducible vertex function, containing all diagrams with insertions of the interaction that can not be separated by cutting a pair of particle-hole single-particle propagators $G G$.

The first order contribution to the vertext $\Gamma$ is the bare interaction $U$ and the approximation

$$ \Gamma \approx U $$

gives the so-called random phase approximation for $\chi$, i.e.

$$ \chi_{RPA} = \chi_0 + \chi_0 U \chi_{RPA} $$

Rewriting this equation gives $\chi_{RPA}$ as

$$ \chi_{RPA} = \frac{\chi_0}{1 - U \chi_0} $$

we note that the denominator of this equation can in general go to zero, whereby the susceptibility $\chi_{RPA}$ diverges. Whence already the RPA approximation can be used to compute instabilities of the system towards, e.g., anti-ferromagnetic symmetry breaking.

As an example we compute $\chi_{RPA}$ for the square lattice and the enhancement of the $k = (\pi, \pi)$ peak as a function of $U$.

In [ ]:
# Pick up chi from HDF5
# RENAME in chi_RPA ?
# Q : write this
# Q : plot chi
# Q : find the instability
from pytriqs.archive import HDFArchive
from pytriqs.gf import Gf

with HDFArchive("chi0.h5",'r') as R:
    chi0_wk = R['chi0_wk']
chi_rpa = chi0_wk.copy()

In [ ]:
wmesh = chi0_wk.mesh[0]
g = Gf(mesh=wmesh, target_shape=[])
g[:] + 1

In [ ]:
from pytriqs.gf import *
chi0_wk[:,Idx(0,0,0)] + 1
#inverse(1. - 2. * chi0_wk[:, Idx(0,0,0)]) * chi0_wk[:, Idx(0,0,0)]

In [ ]:
# The DOUBLE loop will be faster...

def chi_wk_from_U_and_chi0_wk(chi0_wk, U):
    U = float(U)
    chi_wk = chi0_wk.copy()
    kmesh = chi0_wk.mesh[1]
    for k in kmesh:
        chi_wk[:, k] << inverse(1. - U * chi0_wk[:, k]) * chi0_wk[:, k]
    return chi_wk

chi_wk_from_U_and_chi0_wk(chi0_wk, 2)

In [ ]:
from pytriqs.plot.mpl_interface import oplot, oplotr, oploti, plt
from pytriqs.gf import inverse

chi_w0k = chi_wk_from_U_and_chi0_wk(chi0_wk, 2.0)[Idx(0),:]

k = np.linspace(0, 2*np.pi, kmesh.linear_dims[0]+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

chi = lambda kx, ky: chi_w0k([kx,ky,0]).real

plt.figure(figsize=(3.25*2, 5))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.imshow(np.vectorize(chi)(kx,ky))

#plt.imshow(data[:, :, 0].real, cmap=plt.get_cmap('terrain_r'), origin='lower', 
#           vmin=0, extent=extent_k)
plt.colorbar(); plt.tight_layout()
#plt.savefig('figure_chi0_w0k.pdf')
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')


# Here is a 3d plot. With "matplotlib inline" one cannot interact
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 
fig = plt.figure()

ax = Axes3D(fig) #<-- Note the difference from your original code...
ax.plot_surface(kx, ky, np.vectorize(chi)(kx,ky), cmap="jet")
ax.set_xlabel(r"$k_x$")
ax.set_ylabel(r"$k_y$")
ax.set_zlabel(r"$\chi^{0}$")



In [ ]:
from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    chi0_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
chi0_plot = interp(k_path_vecs[:, :2])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')

plt.plot(k_plot, chi0_plot.real, label=r'$\chi_0$')

U_vec = np.arange(0.2, 1.2, 0.2)

for U in U_vec:
    print 'U =', U
    chi_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, U)
    chi = lambda kx, ky: 
    
    k = np.linspace(0, 2*np.pi, kmesh.linear_dims[0]+1, endpoint=True)


    plt.plot(k, chi_plot.real, label=r'$\chi_{RPA}$, $U=%2.2f$' % U)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.legend(loc='best')
plt.savefig('figure_chi0_k_bandpath.pdf')

At some critical $U_c$ the susceptibility diverges $\chi \rightarrow \infty$ within the random phase approximation. To determine $U_c$ we can study the root of the inverse susceptibility $\chi_{RPA}^{-1}$.

For the square lattice it is sufficient to study the response at $k_{AF}= (\pi, \pi)$ since it is the point where the response diverges. Analytically this occurs when the denominator is zero $1 - U_c \chi_0(0, k_{AF}) = 0$, i.e.

$$ U_c^{(RPA)} = \frac{1}{\chi_0(0, k_{AF})} $$

numerically this looks like

In [ ]:
# find kidx where k[kidx] = (pi, pi, 0)
k_vec = np.array([k.value for k in kmesh])
diff = np.linalg.norm(k_vec - np.array([np.pi, np.pi, 0]), axis=1)
kidx_pipi = np.squeeze(np.argwhere(diff < 1.e-8))
k_pipi = np.array([k for k in kmesh])[kidx_pipi]
print 'kidx_pipi =', kidx_pipi
print 'k_pipi =', k_pipi

In [ ]:
chi0_w0kpipi = chi0_wk[Idx(0), k_pipi][0, 0].real

U_c = 1. / chi0_w0kpipi

U_vec = np.arange(0.2, 2.2, 0.2)

chi_vec = np.zeros_like(U_vec)
for idx, U in enumerate(U_vec):
    chi_vec[idx] = chi0_w0kpipi / (1 - U*chi0_w0kpipi)

In [ ]:
plt.figure(figsize=(3.25*3, 8))
plt.plot(U_vec, 1./chi_vec, '.-', label=r'$\chi_{RPA}^{-1}$')
plt.plot(U_vec, 0*U_vec, 'k', lw=0.5)
plt.plot(U_c, 0, 'rs', label=r'$U_c \approx %2.2f$' % U_c, alpha=0.5)
plt.legend(loc='best'); plt.xlim([U_vec.min(), U_vec.max()])
plt.ylabel(r'$\chi_{RPA}^{-1}$'); plt.xlabel(r'$U$')